In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('customer_shopping_behavior.csv')
df.head(5)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

In [4]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [5]:
df.isna().sum()

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

# Step to clean data before making any analysis

- Convert column names into snake case.
- Fill nulls in `review_rating`.
- Change data types.
- Check if `discount_applied` and `promo_code_used` columns are same. If yes delete 'promo_code_used'.
- Check and clean any inconsistencies in categorical values.

In [6]:
# Standardizing column names to snake_case for consistency and readability.

print(df.columns)

df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

df = df.rename(columns={'purchase_amount_(usd)' : 'purchase_amount_USD',
                        'frequency_of_purchases' : 'purchase_frequency'})

print('\nAfter\n')
print(df.columns)

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

After

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount_USD', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'purchase_frequency'],
      dtype='object')


In [7]:
# Filling NaN values in 'review_rating' using the median within each 'category'.

print(f'Total nulls in "review_rating": {df["review_rating"].isna().sum()}')

cateogry_median = df.groupby(['category'])['review_rating'].transform('median')
df['review_rating'] = df['review_rating'].fillna(cateogry_median)

print('\nAfter\n')
print(f'Total nulls in "review_rating": {df["review_rating"].isna().sum()}')

Total nulls in "review_rating": 37

After

Total nulls in "review_rating": 0


In [8]:
# Standardizing column data types for consistency, reduced storage, and improved query performance.

df = df.astype({ 'gender'                 : 'category',
                 'item_purchased'         : 'category',
                 'category'               : 'category',
                 'location'               : 'category',
                 'size'                   : 'category',
                 'color'                  : 'category',
                 'season'                 : 'category',
                 'shipping_type'          : 'category',
                 'payment_method'         : 'category',
                 'purchase_frequency' : 'category'  })

# Converting 'subscription_status','discount_applied','promo_code_used' 
# from categorical ('Yes'/'No') to boolean for efficient storage and logical operations.

for col in ['subscription_status','discount_applied','promo_code_used']:
    df[col] = df[col].map({'Yes' : True,
                           'No'  : False })

# Reduced the file storage from 548.6+ KB to 207.4 KB.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   customer_id          3900 non-null   int64   
 1   age                  3900 non-null   int64   
 2   gender               3900 non-null   category
 3   item_purchased       3900 non-null   category
 4   category             3900 non-null   category
 5   purchase_amount_USD  3900 non-null   int64   
 6   location             3900 non-null   category
 7   size                 3900 non-null   category
 8   color                3900 non-null   category
 9   season               3900 non-null   category
 10  review_rating        3900 non-null   float64 
 11  subscription_status  3900 non-null   bool    
 12  shipping_type        3900 non-null   category
 13  discount_applied     3900 non-null   bool    
 14  promo_code_used      3900 non-null   bool    
 15  previous_purchases   

In [9]:
# Checking if 'discount_applied' and 'promo_code_used' are same or not.

result = (df['discount_applied'] == df['promo_code_used']).all()
print(f"'discount_applied' and 'promo_code_used' columns are same ? : {result}")

# Deleting 'promo_code_used' column.
df = df.drop(columns='promo_code_used')

# Column 'promo_code_used' deleted.
print(f'\n{df.columns.to_list()}')

'discount_applied' and 'promo_code_used' columns are same ? : True

['customer_id', 'age', 'gender', 'item_purchased', 'category', 'purchase_amount_USD', 'location', 'size', 'color', 'season', 'review_rating', 'subscription_status', 'shipping_type', 'discount_applied', 'previous_purchases', 'payment_method', 'purchase_frequency']


In [10]:
# Checking for inconsistencies in categorical values. ( None Found )

for col in df.columns:
    if df[col].dtypes == 'category':
        print(f'\n{col} : {sorted(df[col].unique().tolist())}')
        print(f'Total unique values : {len(df[col].unique().tolist())}\n')


gender : ['Female', 'Male']
Total unique values : 2


item_purchased : ['Backpack', 'Belt', 'Blouse', 'Boots', 'Coat', 'Dress', 'Gloves', 'Handbag', 'Hat', 'Hoodie', 'Jacket', 'Jeans', 'Jewelry', 'Pants', 'Sandals', 'Scarf', 'Shirt', 'Shoes', 'Shorts', 'Skirt', 'Sneakers', 'Socks', 'Sunglasses', 'Sweater', 'T-shirt']
Total unique values : 25


category : ['Accessories', 'Clothing', 'Footwear', 'Outerwear']
Total unique values : 4


location : ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermon

#  Feature Engineering

In [11]:
# Creating 'age_group' column from 'age'.

labels = ['Young Adult','Adult','Mid-aged','Senior']
df['age_group'] = pd.qcut(df['age'], q=4, labels=labels)

df['age_group'].head(10)

0       Mid-aged
1    Young Adult
2       Mid-aged
3    Young Adult
4       Mid-aged
5       Mid-aged
6         Senior
7    Young Adult
8    Young Adult
9       Mid-aged
Name: age_group, dtype: category
Categories (4, object): ['Young Adult' < 'Adult' < 'Mid-aged' < 'Senior']

In [12]:
# Creating 'purchase_frequency_days' column from 'purchase_frequency'.

frq_days_map = { 'Annually'       : 365,
             'Bi-Weekly'      : 14,
             'Every 3 Months' : 90,
             'Fortnightly'    : 14,
             'Monthly'        : 30,
             'Quarterly'      : 90,
             'Weekly'         : 7}

df['purchase_frequency_days'] = df['purchase_frequency'].map(frq_days_map)

df['purchase_frequency_days'].head(15)

0      14
1      14
2       7
3       7
4     365
5       7
6      90
7       7
8     365
9      90
10     14
11     14
12     14
13      7
14      7
Name: purchase_frequency_days, dtype: int64

In [13]:
df.head(5)

,customer_id,age,gender,item_purchased,category,purchase_amount_USD,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,purchase_frequency,age_group,purchase_frequency_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,True,Express,True,14,Venmo,Fortnightly,Mid-aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,True,Express,True,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,True,Free Shipping,True,23,Credit Card,Weekly,Mid-aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,True,Next Day Air,True,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,True,Free Shipping,True,31,PayPal,Annually,Mid-aged,365


# Loading data to MySQL

In [14]:
from sqlalchemy import create_engine

username = 'root'
password = '741'
host = 'localhost'
port = '3306'
database = 'customer_shopping_behaviour'

# Creating engine
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# Uploading DataFrame (replace if table exists)
table_name = 'customer_shopping_behaviour_table'
df.to_sql(table_name, engine, if_exists="replace", index=False)

# Verifying upload
pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5", engine)

,customer_id,age,gender,item_purchased,category,purchase_amount_USD,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,purchase_frequency,age_group,purchase_frequency_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,1,Express,1,14,Venmo,Fortnightly,Mid-aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,1,Express,1,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,1,Free Shipping,1,23,Credit Card,Weekly,Mid-aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,1,Next Day Air,1,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,1,Free Shipping,1,31,PayPal,Annually,Mid-aged,365
